In [1]:
import chess
import time
from itertools import chain
from collections import defaultdict

# --- Database Imports ---
# Assuming these imports are correctly configured for your database setup
# and provide the necessary models and DBInterface class.
from database.database.ask_db import open_request, get_ask_connection
from database.operations.collect_fens import get_new_games_links
from database.operations.models import (
    
    RawfenCreateData, # Pydantic model for Rawfen creation data
    KnownfensCreateData, # Pydantic model for Knownfens creation data
    
)

from database.database.models import (Rawfen,
                                        Knownfens,to_dict)
from database.database.db_interface import DBInterface # Your DBInterface class
from database.database.engine import init_db
from constants import CONN_STRING
init_db(CONN_STRING)
# --- Helper function: Generates FENs for a single game's moves ---
def _generate_fens_for_single_game_moves(moves: list[dict]) -> list[str]:
    """
    Generates a sequence of FENs for a single chess game given its moves.

    Args:
        moves (list[dict]): A list of dictionaries, where each dictionary represents
                            a move with keys 'n_move', 'white_move', 'black_move'.
                            Example: [{'n_move': 1, 'white_move': 'e4', 'black_move': 'e5'}]

    Returns:
        list[str]: A list of FEN strings representing the board state after each half-move.
                   Returns a partial list or an empty list if an invalid move is encountered,
                   along with a printed error message.
    """
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    
    for ind, move in enumerate(moves):
        # Basic assertion to check move order consistency, can be removed if not critical
        if move['n_move'] != ind + 1:
            print(f"Warning: n_move mismatch for move {move['n_move']} at index {ind}. Expected {ind + 1}. "
                  f"Processing might be out of order for this game.")
            # Decide if you want to continue or stop here based on data integrity needs.
            # For now, we continue but warn.
            
        n_move = move['n_move']
        white_move_san = move.get('white_move') # Use .get() for safer access
        black_move_san = move.get('black_move') # Use .get() for safer access

        # Apply White's move and get FEN
        if white_move_san: # Only attempt if white_move exists
            try:
                move_obj_white = board.parse_san(white_move_san)
                board.push(move_obj_white)
                fens_sequence.append(board.fen())
            except (ValueError, chess.InvalidMoveError) as e:
                print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
                # Stop processing this game if an invalid move is found
                return fens_sequence
        
        # Apply Black's move and get FEN (only if black_move exists and white's move was successful)
        if black_move_san: # Only attempt if black_move exists
            try:
                move_obj_black = board.parse_san(black_move_san)
                board.push(move_obj_black)
                fens_sequence.append(board.fen())
            except (ValueError, chess.InvalidMoveError) as e:
                print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
                # Stop processing this game if an invalid move is found
                return fens_sequence
                
    return fens_sequence

# --- Optimized Database Fetching Function ---
def get_all_moves_for_links_batch(game_links: list[str]) -> dict[str, list[dict]]:
    """
    Fetches all moves for a given list of game links in a single batched query
    to the database.

    Args:
        game_links (list[str]): A list of game links (URLs or unique identifiers).

    Returns:
        dict[str, list[dict]]: A dictionary where keys are game links and values are lists of
                               move dictionaries, sorted by 'n_move' for each game.
                               Returns an empty dictionary if no links are provided or no
                               moves are found.
    """
    if not game_links:
        return {}

    # SQL query to select moves for all provided links.
    # Using UNNEST for the array parameter is efficient for PostgreSQL.
    # IMPORTANT FIX: Added ::bigint cast to ensure type compatibility with 'link' column.
    sql_query = """
    SELECT link, n_move, white_move, black_move
    FROM moves
    WHERE link IN (SELECT unnest(%s::text[])::bigint)
    ORDER BY link, n_move;
    """
    
    # open_request is assumed to execute the query with parameterized input
    # and return a list of tuples (link, n_move, white_move, black_move).
    result_tuples = open_request(sql_query, params=(game_links,))

    # Group the fetched moves by game link for easier processing
    grouped_moves = defaultdict(list)
    for row in result_tuples:
        link, n_move, white_move, black_move = row
        grouped_moves[link].append({
            'n_move': n_move,
            'white_move': white_move,
            'black_move': black_move
        })
    return grouped_moves

# --- Main FEN Generation Orchestrator (Optimized) ---
def get_fens_from_games_optimized(new_game_links_data: list[tuple]) -> list[str]:
    """
    Retrieves and generates unique FENs for a list of game links.
    This version is optimized to fetch all game moves in a single batched query
    to significantly reduce execution time.

    Args:
        new_game_links_data (list[tuple]): A list of tuples, where each tuple's
                                           first element is a game link.
                                           Typically, this comes from get_new_games_links.

    Returns:
        list[str]: A unique list of FEN strings generated from all processed games.
    """
    all_fens = set() # Use a set for efficient storage of unique FENs
    game_links_only = [x[0] for x in new_game_links_data] # Extract links from the input tuples

    start_db_fetch = time.time()
    # Fetch all moves for all games in a single batched query
    all_game_moves_grouped = get_all_moves_for_links_batch(game_links_only)
    db_fetch_time = time.time() - start_db_fetch
    print(f"Time to fetch all game moves from DB (batched): {db_fetch_time:.4f} seconds")

    total_fen_generation_time = 0
    games_processed = 0
    # Process each game's moves to generate FENs
    for game_link in game_links_only: # Iterate through the original list to ensure all links are attempted
        game_moves = all_game_moves_grouped.get(game_link)
        
        if game_moves:
            fen_gen_start = time.time()
            try:
                # Use the renamed helper function for single game FEN generation
                game_fens = _generate_fens_for_single_game_moves(game_moves)
                all_fens.update(game_fens) # Add FENs to the set
                games_processed += 1
            except Exception as e: # Catch any unexpected errors during FEN generation
                print(f"An unexpected error occurred while processing game {game_link}: {e}")
                # Continue to the next game even if one game fails
            total_fen_generation_time += (time.time() - fen_gen_start)
        else:
            print(f"No moves found in the database for game link: {game_link}")

    if games_processed > 0:
        print(f"Mean FEN generation time per game (excluding DB fetch): {total_fen_generation_time / games_processed:.4f} seconds")
    
    return list(all_fens) # Convert the set back to a list for the final output

# --- Function to get new FENs (from your original code, remains unchanged) ---
def get_new_fens(posible_fens: list[str]) -> list[str]:
    """
    Compares a list of possible FENs against known FENs in the 'rawfen' table
    and returns only the FENs that are not already present.

    Args:
        posible_fens (list[str]): A list of FEN strings to check.

    Returns:
        list[str]: A list of FEN strings that are new (not in 'rawfen').
    """
    if not posible_fens:
        return []

    sql_query = """
    SELECT p_fen.f
    FROM UNNEST(%s::text[]) AS p_fen(f)
    WHERE p_fen.f NOT IN (SELECT fen FROM rawfen);
    """
    # open_request is assumed to handle the query and return results.
    result_tuples = open_request(sql_query, params=(posible_fens,))
    valid_fens = list(chain.from_iterable(result_tuples))
    return valid_fens

# --- Functions for Inserting Data (Adjusted for your DBInterface) ---
def insert_fens(fens: list[str]):
    """
    Inserts a list of new FENs into the rawfen table.
    """
    try:
        to_insert_fens = [RawfenCreateData(**{'fen':x}).model_dump() for x in fens]
        rawfen_interface = DBInterface(Rawfen) # Removed 'session=session'
        rawfen_interface.create_all(to_insert_fens)
        print(f"Successfully inserted {len(fens)} FENs.")
    except Exception as e:
        print(f"Error inserting FENs: {e}")
        # The DBInterface.create_all method should handle internal rollback/commit/close
        # We re-raise if you want the outer script to know about the failure.
        raise

def insert_games(links: list[tuple]):
    """
    Inserts a list of game links into the knownfens table.
    """
    try:
        to_insert_games = [KnownfensCreateData(**{'link':x[0]}).model_dump() for x in links]
        game_interface = DBInterface(Knownfens) # Removed 'session=session'
        game_interface.create_all(to_insert_games)
        print(f"Successfully inserted {len(links)} game links.")
    except Exception as e:
        print(f"Error inserting game links: {e}")
        # The DBInterface.create_all method should handle internal rollback/commit/close
        # We re-raise if you want the outer script to know about the failure.
        raise

# --- Execution Flow ---
print("Fetching new game links...")
# You can change the number of links to fetch here, e.g., get_new_games_links(10000)
new_game_links = get_new_games_links(100) 

print(f"Retrieved {len(new_game_links)} new game links.")

start_total_fen_gen = time.time()
fen_set_from_games = get_fens_from_games_optimized(new_game_links)
print('Total time for fen_set_from_games (optimized): ', time.time() - start_total_fen_gen)
print(f"Generated {len(fen_set_from_games)} unique FENs.")

start_new_fens_check = time.time()
new_fens = get_new_fens(fen_set_from_games)
print('Time for get_new_fens: ', time.time() - start_new_fens_check)
print(f"Found {len(new_fens)} genuinely new FENs.")

# --- Inserting FENs and Games into the Database ---
print("\n--- Inserting data into the database ---")
start_insert_fens = time.time()
insert_fens(new_fens)
print('insert_fens time elapsed: ', time.time() - start_insert_fens)

start_insert_games = time.time()
insert_games(new_game_links)
print('insert_games time elapsed: ', time.time() - start_insert_games)


/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Fetching new game links...
Retrieved 100 new game links.
Time to fetch all game moves from DB (batched): 2.0629 seconds
Mean FEN generation time per game (excluding DB fetch): 0.0033 seconds
Total time for fen_set_from_games (optimized):  2.3960416316986084
Generated 7097 unique FENs.
Time for get_new_fens:  0.03801226615905762
Found 7097 genuinely new FENs.

--- Inserting data into the database ---
Successfully inserted 7097 FENs.
insert_fens time elapsed:  0.2503941059112549
Successfully inserted 100 game links.
insert_games time elapsed:  0.0055217742919921875


In [1]:
import chess
import time
from itertools import chain
from collections import defaultdict

# Assuming these imports are correctly configured for your database setup
from database.database.ask_db import open_request, get_ask_connection
from database.operations.collect_fens import get_new_games_links
from database.database.models import to_dict # Kept for compatibility, though not used in this snippet
from database.operations.models import KnownfensCreateData, RawfenCreateData
from database.database.db_interface import DBInterface
from database.database.models import Knownfens, Rawfen

/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:

# --- Helper function: Generates FENs for a single game's moves ---
# Renamed from the original 'get_fens_from_games' to avoid ambiguity and indicate its role.
def _generate_fens_for_single_game_moves(moves: list[dict]) -> list[str]:
    """
    Generates a sequence of FENs for a single chess game given its moves.

    Args:
        moves (list[dict]): A list of dictionaries, where each dictionary represents
                            a move with keys 'n_move', 'white_move', 'black_move'.
                            Example: [{'n_move': 1, 'white_move': 'e4', 'black_move': 'e5'}]

    Returns:
        list[str]: A list of FEN strings representing the board state after each half-move.
                   Returns a partial list or an empty list if an invalid move is encountered,
                   along with a printed error message.
    """
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    
    for ind, move in enumerate(moves):
        # Basic assertion to check move order consistency, can be removed if not critical
        if move['n_move'] != ind + 1:
            print(f"Warning: n_move mismatch for move {move['n_move']} at index {ind}. Expected {ind + 1}. "
                  f"Processing might be out of order for this game.")
            # Decide if you want to continue or stop here based on data integrity needs.
            # For now, we continue but warn.
            
        n_move = move['n_move']
        white_move_san = move.get('white_move') # Use .get() for safer access
        black_move_san = move.get('black_move') # Use .get() for safer access

        # Apply White's move and get FEN
        if white_move_san: # Only attempt if white_move exists
            try:
                move_obj_white = board.parse_san(white_move_san)
                board.push(move_obj_white)
                fens_sequence.append(board.fen())
            except (ValueError, chess.InvalidMoveError) as e:
                print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
                # Stop processing this game if an invalid move is found
                return fens_sequence
        
        # Apply Black's move and get FEN (only if black_move exists and white's move was successful)
        if black_move_san: # Only attempt if black_move exists
            try:
                move_obj_black = board.parse_san(black_move_san)
                board.push(move_obj_black)
                fens_sequence.append(board.fen())
            except (ValueError, chess.InvalidMoveError) as e:
                print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
                # Stop processing this game if an invalid move is found
                return fens_sequence
                
    return fens_sequence

# --- Optimized Database Fetching Function ---
def get_all_moves_for_links_batch(game_links: list[str]) -> dict[str, list[dict]]:
    """
    Fetches all moves for a given list of game links in a single batched query
    to the database.

    Args:
        game_links (list[str]): A list of game links (URLs or unique identifiers).

    Returns:
        dict[str, list[dict]]: A dictionary where keys are game links and values are lists of
                               move dictionaries, sorted by 'n_move' for each game.
                               Returns an empty dictionary if no links are provided or no
                               moves are found.
    """
    if not game_links:
        return {}

    # SQL query to select moves for all provided links.
    # Using UNNEST for the array parameter is efficient for PostgreSQL.
    # IMPORTANT FIX: Added ::bigint cast to ensure type compatibility with 'link' column.
    sql_query = """
    SELECT link, n_move, white_move, black_move
    FROM moves
    WHERE link IN (SELECT unnest(%s::text[])::bigint)
    ORDER BY link, n_move;
    """
    
    # open_request is assumed to execute the query with parameterized input
    # and return a list of tuples (link, n_move, white_move, black_move).
    result_tuples = open_request(sql_query, params=(game_links,))

    # Group the fetched moves by game link for easier processing
    grouped_moves = defaultdict(list)
    for row in result_tuples:
        link, n_move, white_move, black_move = row
        grouped_moves[link].append({
            'n_move': n_move,
            'white_move': white_move,
            'black_move': black_move
        })
    return grouped_moves

# --- Main FEN Generation Orchestrator (Optimized) ---
# This function replaces your second 'get_fens_from_games' definition.
def get_fens_from_games_optimized(new_game_links_data: list[tuple]) -> list[str]:
    """
    Retrieves and generates unique FENs for a list of game links.
    This version is optimized to fetch all game moves in a single batched query
    to significantly reduce execution time.

    Args:
        new_game_links_data (list[tuple]): A list of tuples, where each tuple's
                                           first element is a game link.
                                           Typically, this comes from get_new_games_links.

    Returns:
        list[str]: A unique list of FEN strings generated from all processed games.
    """
    all_fens = set() # Use a set for efficient storage of unique FENs
    game_links_only = [x[0] for x in new_game_links_data] # Extract links from the input tuples

    start_db_fetch = time.time()
    # Fetch all moves for all games in a single batched query
    all_game_moves_grouped = get_all_moves_for_links_batch(game_links_only)
    db_fetch_time = time.time() - start_db_fetch
    print(f"Time to fetch all game moves from DB (batched): {db_fetch_time:.4f} seconds")

    total_fen_generation_time = 0
    games_processed = 0
    # Process each game's moves to generate FENs
    for game_link in game_links_only: # Iterate through the original list to ensure all links are attempted
        game_moves = all_game_moves_grouped.get(game_link)
        
        if game_moves:
            fen_gen_start = time.time()
            try:
                # Use the renamed helper function for single game FEN generation
                game_fens = _generate_fens_for_single_game_moves(game_moves)
                all_fens.update(game_fens) # Add FENs to the set
                games_processed += 1
            except Exception as e: # Catch any unexpected errors during FEN generation
                print(f"An unexpected error occurred while processing game {game_link}: {e}")
                # Continue to the next game even if one game fails
            total_fen_generation_time += (time.time() - fen_gen_start)
        else:
            print(f"No moves found in the database for game link: {game_link}")

    if games_processed > 0:
        print(f"Mean FEN generation time per game (excluding DB fetch): {total_fen_generation_time / games_processed:.4f} seconds")
    
    return list(all_fens) # Convert the set back to a list for the final output

# --- Function to get new FENs (from your original code, remains unchanged) ---
def get_new_fens(posible_fens: list[str]) -> list[str]:
    """
    Compares a list of possible FENs against known FENs in the 'rawfen' table
    and returns only the FENs that are not already present.

    Args:
        posible_fens (list[str]): A list of FEN strings to check.

    Returns:
        list[str]: A list of FEN strings that are new (not in 'rawfen').
    """
    if not posible_fens:
        return []

    sql_query = """
    SELECT p_fen.f
    FROM UNNEST(%s::text[]) AS p_fen(f)
    WHERE p_fen.f NOT IN (SELECT fen FROM rawfen);
    """
    # open_request is assumed to handle the query and return results.
    result_tuples = open_request(sql_query, params=(posible_fens,))
    valid_fens = list(chain.from_iterable(result_tuples))
    return valid_fens

# --- Execution Flow (as in your original notebook cell) ---
# This part calls the functions with the new optimized versions.
print("Fetching new game links...")
new_game_links = get_new_games_links(10000) # This line remains as per your request

print(f"Retrieved {len(new_game_links)} new game links.")

start_total = time.time()
# Call the optimized function for FEN generation
fen_set_from_games = get_fens_from_games_optimized(new_game_links)
print('Total time for fen_set_from_games (optimized): ', time.time() - start_total)
print(f"Generated {len(fen_set_from_games)} unique FENs.")

start_new_fens = time.time()
new_fens = get_new_fens(fen_set_from_games)
print('Time for get_new_fens: ', time.time() - start_new_fens)
print(f"Found {len(new_fens)} genuinely new FENs.")

Fetching new game links...
Retrieved 10000 new game links.
Time to fetch all game moves from DB (batched): 1.4365 seconds
Mean FEN generation time per game (excluding DB fetch): 0.0034 seconds
Total time for fen_set_from_games (optimized):  35.195550203323364
Generated 666509 unique FENs.
Time for get_new_fens:  1.9511029720306396
Found 666509 genuinely new FENs.


In [3]:
def insert_fens(fens):
    to_insert_fens = [RawfenCreateData(**{'fen':x}).model_dump() for x in fens]
    rawfen_interface = DBInterface(Rawfen)
    rawfen_interface.create_all(to_insert_fens)
def insert_games(links):
    to_insert_fens = [KnownfensCreateData(**{'link':x[0]}).model_dump() for x in links]
    game_interface = DBInterface(Knownfens)
    game_interface.create_all(to_insert_fens)

In [4]:
start = time.time()
insert_fens(new_fens)
print('insert_fens time elapsed: ', time.time()-start)
start = time.time()
insert_games(new_game_links)
print('insert_games time elapsed: ', time.time()-start)

UnboundExecutionError: Could not locate a bind configured on mapper Mapper[Rawfen(rawfen)] or this Session.

In [4]:
from database.database.ask_db import *
from database.operations.collect_fens import get_new_games_links#, get_fens_from_games, get_fens_from_games
from database.database.models import to_dict
import chess
import time

new_game_links = get_new_games_links(100)

start = time.time()
def get_fens_from_games(moves):
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    for ind, move in enumerate(moves):
        assert move['n_move'] == ind+1
        n_move = move['n_move']
        white_move_san = move['white_move']
        black_move_san = move['black_move']
        current_fens = {'n_move': n_move}
        
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        #fens_sequence.append(current_fens)
    return fens_sequence
def get_one_game(link):
    conn = get_ask_connection()
    try:
        with conn.cursor() as curs:
            curs.execute(f"select moves.n_move, white_move, black_move from moves where moves.link = '{link}'")
            column_names = [desc[0] for desc in curs.description]
            results = []
            for row in curs.fetchall():
                results.append(dict(zip(column_names, row)))
            return results
    finally:
        conn.close()
def get_fens_from_games(new_game_links):
    fens = []
    new_game_links = [x[0] for x in new_game_links]
    one_game_move_analisys_list = []
    for game_link in new_game_links:
        one_game_move_analisys = time.time()
        game_moves = get_one_game(game_link)
        one_game_move_analisys_list.append(time.time()-one_game_move_analisys)
        try:
            fens.extend(get_fens_from_moves(game_moves))
        except:
            continue
            #i don't want to call numpy just for the next line
    print("one_game_analysis_mean: ",sum(one_game_move_analisys_list)/len(one_game_move_analisys_list))
    return list(set(fens))
fen_set_from_games = get_fens_from_games(new_game_links)
print('fen_set_from_games: ', time.time()-start)

start = time.time()

def get_new_fens(posible_fens: list[str]) -> list[str]:
    if not posible_fens:
        return []

    sql_query = """
    SELECT p_fen.f
    FROM UNNEST(%s::text[]) AS p_fen(f)  -- 'p_fen' is an alias for the unnested array, 'f' is the column name
    WHERE p_fen.f NOT IN (SELECT fen FROM rawfen); -- FIX IS HERE: Changed 'known_fens' to 'fen' and 'link' to 'fen'
    """
    result_tuples = open_request(sql_query, params=(posible_fens,))
    valid_fens = list(chain.from_iterable(result_tuples))
    return valid_fens
new_fens = get_new_fens(fen_set_from_games)
print('new_fens: ', time.time()-start)

one_game_analysis_mean:  0.252740261554718
fen_set_from_games:  25.275590896606445
new_fens:  0.00011134147644042969


In [ ]:
def get_fens_from_games(moves):
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    for ind, move in enumerate(moves):
        assert move['n_move'] == ind+1
        n_move = move['n_move']
        white_move_san = move['white_move']
        black_move_san = move['black_move']
        current_fens = {'n_move': n_move}
        
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        #fens_sequence.append(current_fens)
    return fens_sequence
fen_set_from_games = get_fens_from_games(new_game_links)

In [10]:
get_only_new_fens = get_new_fens(fen_set_from_games)

In [11]:
new_fens = get_new_fens(fen_set_from_games)

In [ ]:
insert_fens(new_fens)

In [ ]:
insert_games(new_game_links)

In [ ]:
fen_set_from_games = get_fens_from_games(new_game_links)

In [23]:
a = get_one_game(valid[0][0])

In [25]:
def get_fens_from_games(moves):
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []
    for ind, move in enumerate(moves):
        assert move['n_move'] == ind+1
        n_move = move['n_move']
        white_move_san = move['white_move']
        black_move_san = move['black_move']
        current_fens = {'n_move': n_move}
        
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            fens_sequence.append(board.fen())
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        #fens_sequence.append(current_fens)
    return fens_sequence

In [26]:
a = get_fens_from_games(a)

In [27]:
a

['rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
 'rnbqkbnr/pp1ppppp/2p5/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 0 2',
 'rnbqkbnr/pp1ppppp/2p5/8/2PP4/8/PP2PPPP/RNBQKBNR b KQkq - 0 2',
 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3',
 'rnbqkbnr/pp2pppp/2p5/3p4/2PP4/5N2/PP2PPPP/RNBQKB1R b KQkq - 1 3',
 'rnbqkb1r/pp2pppp/2p2n2/3p4/2PP4/5N2/PP2PPPP/RNBQKB1R w KQkq - 2 4',
 'rnbqkb1r/pp2pppp/2p2n2/3p4/2PP4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 3 4',
 'rnbqkb1r/pp3ppp/2p1pn2/3p4/2PP4/2N2N2/PP2PPPP/R1BQKB1R w KQkq - 0 5',
 'rnbqkb1r/pp3ppp/2p1pn2/3P4/3P4/2N2N2/PP2PPPP/R1BQKB1R b KQkq - 0 5',
 'rnbqkb1r/pp3ppp/2p2n2/3p4/3P4/2N2N2/PP2PPPP/R1BQKB1R w KQkq - 0 6',
 'rnbqkb1r/pp3ppp/2p2n2/3p2B1/3P4/2N2N2/PP2PPPP/R2QKB1R b KQkq - 1 6',
 'rnbqk2r/pp3ppp/2pb1n2/3p2B1/3P4/2N2N2/PP2PPPP/R2QKB1R w KQkq - 2 7',
 'rnbqk2r/pp3ppp/2pb1n2/3p2B1/3P4/2N2N2/PPQ1PPPP/R3KB1R b KQkq - 3 7',
 'rnbqk2r/pp3pp1/2pb1n1p/3p2B1/3P4/2N2N2/PPQ1PPPP/R3KB1R w KQkq - 0 8',
 'rnbqk2r/pp3pp1/2pb1n1p/3p4/3P3B

In [ ]:
def get_fens_for_game_moves(moves_list: list[dict]) -> list[dict]:
    """
    Generates FENs for each half-move in a list of chess moves.

    Args:
        moves_list: A list of dictionaries, where each dictionary has
                    'n_move', 'white_move' (SAN), and 'black_move' (SAN).

    Returns:
        A list of dictionaries, each containing:
        - 'n_move': The move number.
        - 'white_fen': The FEN string after White's move.
        - 'black_fen': The FEN string after Black's move.
        Returns an empty list or stops processing if an invalid move is encountered.
    """
    board = chess.Board() # Initialize a standard chess board
    fens_sequence = []

    for move_entry in moves_list:
        n_move = move_entry['n_move']
        white_move_san = move_entry['white_move']
        black_move_san = move_entry['black_move']

        current_fens = {'n_move': n_move}

        # 1. Apply White's move and get FEN
        try:
            move_obj_white = board.parse_san(white_move_san)
            board.push(move_obj_white)
            current_fens['white_fen'] = board.fen()
        except ValueError as e:
            print(f"Error applying White's move '{white_move_san}' at move number {n_move}: {e}")
            # If a move is invalid, the sequence is broken. You might want to
            # return what you have so far or raise an error depending on your needs.
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid White's move '{white_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error


        # 2. Apply Black's move and get FEN
        try:
            move_obj_black = board.parse_san(black_move_san)
            board.push(move_obj_black)
            current_fens['black_fen'] = board.fen()
        except ValueError as e:
            print(f"Error applying Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error
        except chess.InvalidMoveError as e:
            print(f"Invalid Black's move '{black_move_san}' at move number {n_move}: {e}")
            return fens_sequence # Stop processing on error

        fens_sequence.append(current_fens)

    return fens_sequence

# Get the FENs for your list of moves
game_fens_result = get_fens_for_game_moves(moves_data)

# Print the results
for entry in game_fens_result:
    print(f"Move {entry['n_move']}:")
    print(f"  White FEN: {entry['white_fen']}")
    print(f"  Black FEN: {entry['black_fen']}")

# You can also verify the final board state if needed
if game_fens_result:
    print(f"\nFinal FEN after all moves: {game_fens_result[-1]['black_fen']}")

In [14]:
a

[{'n_move': 1, 'white_move': 'd4', 'black_move': 'c6'},
 {'n_move': 2, 'white_move': 'c4', 'black_move': 'd5'},
 {'n_move': 3, 'white_move': 'Nf3', 'black_move': 'Nf6'},
 {'n_move': 4, 'white_move': 'Nc3', 'black_move': 'e6'},
 {'n_move': 5, 'white_move': 'cxd5', 'black_move': 'exd5'},
 {'n_move': 6, 'white_move': 'Bg5', 'black_move': 'Bd6'},
 {'n_move': 7, 'white_move': 'Qc2', 'black_move': 'h6'},
 {'n_move': 8, 'white_move': 'Bh4', 'black_move': 'O-O'},
 {'n_move': 9, 'white_move': 'e3', 'black_move': 'Bg4'},
 {'n_move': 10, 'white_move': 'Bd3', 'black_move': 'Nbd7'},
 {'n_move': 11, 'white_move': 'h3', 'black_move': 'Be6'},
 {'n_move': 12, 'white_move': 'O-O', 'black_move': 'Qc7'},
 {'n_move': 13, 'white_move': 'Rad1', 'black_move': 'g5'},
 {'n_move': 14, 'white_move': 'Bxg5', 'black_move': 'hxg5'},
 {'n_move': 15, 'white_move': 'Nxg5', 'black_move': 'Kg7'},
 {'n_move': 16, 'white_move': 'Bf5', 'black_move': 'Bxf5'},
 {'n_move': 17, 'white_move': 'Qxf5', 'black_move': 'Rh8'},
 {'n_m